# Исследование текущего уровня потребительской лояльности среди клиентов из России

***Заказчик этого исследования*** — большая телекоммуникационная компания, которая оказывает услуги на территории всего СНГ. Перед компанией стоит задача определить текущий уровень потребительской лояльности, или NPS (от англ. Net Promoter Score), среди клиентов из России.

Чтобы определить уровень лояльности, клиентам задавали классический вопрос: «Оцените по шкале от 1 до 10 вероятность того, что вы порекомендуете компанию друзьям и знакомым».

Компания провела опрос и попросила вас подготовить дашборд с его итогами. Большую базу данных для такой задачи разворачивать не стали и выгрузили данные в SQLite.

Чтобы оценить результаты опроса, оценки обычно делят на три группы:
9-10 баллов — «cторонники» (англ. promoters);
7-8 баллов — «нейтралы» (англ. passives);
0-6 баллов — «критики» (англ. detractors).


Для того чтобы выгрузить данные импортируем необходимые библиотеки и подключимся к базе данных.

In [1]:
import pandas as pd
import numpy as np

from sqlalchemy import create_engine

In [2]:
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

С помощью запроса выгрузим из базы данных необходимые для анализа таблицы и столбцы. Сразу же в запросе преобразуем столбцы в нужные категории, такие как новые и старые пользователи, и группы пользователей по результатам опроса

In [3]:
query = """
SELECT u.user_id,
       u.lt_day,
       CASE
          WHEN u.lt_day > 365 THEN 'old'
          WHEN u.lt_day <= 365 THEN 'new'
       END as is_new,
       u.age,
       u.gender_segment,
       u.os_name,
       u.cpe_type_name,
       l.country,
       l.city,
       CASE 
          WHEN LENGTH(ags.title) > 6 
          THEN substr(ags.title,4,5) 
          ELSE ags.title 
          END as age_segment,
       CASE 
          WHEN LENGTH(ts.title) > 5 
          THEN substr(ts.title,4,5) 
          ELSE ts.title
          END as traffic_segment,
       CASE 
          WHEN LENGTH(ls.title) > 2 
          THEN substr(ls.title,4,5) 
          ELSE ls.title
          END as lifetime_segment,
       u.nps_score,
       CASE 
          WHEN u.nps_score = 9 or u.nps_score = 10 THEN 'promoters'
          WHEN u.nps_score >= 7 and u.nps_score <= 8 THEN 'passives'
          WHEN u.nps_score <= 6 THEN 'detractors'
       END as nps_group
FROM user as u
JOIN location as l on u.location_id = l.location_id
JOIN age_segment as ags on u.age_gr_id = ags.age_gr_id
JOIN traffic_segment as ts on u.tr_gr_id = ts.tr_gr_id
JOIN lifetime_segment as ls on u.lt_gr_id = ls.lt_gr_id
"""


Проверим, что запрос сработал корректно

In [4]:
user = pd.read_sql(query, engine)
user.tail()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
502488,ZZZKLD,1249,old,54.0,1.0,ANDROID,SMARTPHONE,Россия,Москва,45-54,1-5,36+,5,detractors
502489,ZZZLWY,129,new,31.0,0.0,ANDROID,SMARTPHONE,Россия,Москва,25-34,1-5,4-6,8,passives
502490,ZZZQ5F,522,old,36.0,0.0,ANDROID,SMARTPHONE,Россия,Сургут,35-44,25-30,13-24,10,promoters
502491,ZZZQ8E,2936,old,37.0,1.0,ANDROID,SMARTPHONE,Россия,УланУдэ,35-44,65-70,36+,9,promoters
502492,ZZZYH0,2471,old,37.0,0.0,IOS,SMARTPHONE,Россия,Самара,35-44,10-15,36+,2,detractors


Выгружаем файл csv в необходимое нам место

In [5]:
user.to_csv('telecomm_csi_tableau.csv', index=False)

Результаты исследования представлены на дашборде и презентации

### Укажите ссылку на дашборд на сайте Tableau Public:

https://public.tableau.com/app/profile/eugene8827/viz/Dashboardfortelecommunicationcompany/Dashboard?publish=yes

### Укажите ссылку на pdf-файл с презентацией:

https://drive.google.com/file/d/1YOSr4uxjqvFRvKZryy4o5dpdPgzzRd_O/view?usp=sharing